# Final Project 

This is my final project.  As with previous regression assignments with this dataset, 

In [1]:
import pandas as pd
import numpy as np

In [2]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html
# Various parts of the pandas api reference have been used, will not be citing every specific page.
raw_data = pd.read_csv('survey_results_public.csv', index_col="ResponseId")
raw_data.shape

(65437, 113)

# Analysis of developers who didn't learn to code in college

In [27]:
# Only using columns that I find relevant for the question I want to ask
data = raw_data.filter(items=[
    "MainBranch",
    "Employment",
    "EdLevel",
    "LearnCode",
    "Country",
    "ConvertedCompYearly",
    "JobSat"
])

# Filtering results for full time, employed developers in the US
data["Country"] = data["Country"] == "United States of America"
data["MainBranch"] = data["MainBranch"] == "I am a developer by profession"
data["Employment"] = data["Employment"] == "Employed, full-time"
data["LearnCode"] = data["LearnCode"].str.contains("School")

# Removing NA values
data.dropna(inplace=True, subset=[
    "MainBranch",
    "Employment",
    "EdLevel",
    "LearnCode",
    "Country",
    "ConvertedCompYearly",
    "JobSat"
])

# Dollar amount is poverty line for a 4 person family
data["Successful"] = (data["JobSat"] > 5) & (data["ConvertedCompYearly"] > 32150)

# https://stackoverflow.com/questions/17071871/how-do-i-select-rows-from-a-dataframe-based-on-column-values
data = data.loc[data["Country"] & data["MainBranch"] & data["Employment"]]
data = data.filter(items=[
#    "EdLevel",
    "LearnCode",
#    "JobSat"
    "Successful"
])

data.dropna(inplace=True)

data.describe()


,LearnCode,Successful
count,2631,2631
unique,2,2
top,True,True
freq,1425,2118


In [28]:
data.value_counts()

LearnCode  Successful
True       True          1165
False      True           953
True       False          260
False      False          253
Name: count, dtype: int64

This data shows that roughly 80% of software developers are considered "successful" by my very limited classification.  More importantly, however, this shows that the proportion is roughly the same, regardless of whether you learned to code in college.  

This shows that college is not the only path to success. If it doesn't work for you, then you don't have to go to college to be a successful developer.  

# Analysis of Developers in different industries

In [30]:
# Only using columns that I find relevant for the question I want to ask
data = raw_data.filter(items=[
    "MainBranch",
    "Employment",
    "EdLevel",
    "Country",
    "Industry",
    "ConvertedCompYearly",
    "JobSat"
])

# Filtering results for full time, employed developers in the US
data["Country"] = data["Country"] == "United States of America"
data["MainBranch"] = data["MainBranch"] == "I am a developer by profession"
data["Employment"] = data["Employment"] == "Employed, full-time"

# Removing NA values
data.dropna(inplace=True, subset=[
    "MainBranch",
    "Employment",
    "EdLevel",
    "Country",
    "Industry",
    "ConvertedCompYearly",
    "JobSat"
])

# Dollar amount is poverty line for a 4 person family
data["Successful"] = (data["JobSat"] > 5) & (data["ConvertedCompYearly"] > 32150)

# https://stackoverflow.com/questions/17071871/how-do-i-select-rows-from-a-dataframe-based-on-column-values
data = data.loc[data["Country"] & data["MainBranch"] & data["Employment"]]

data = data.filter(items=[
    "Industry",
    "Successful"
])

# https://stackoverflow.com/questions/32387266/converting-categorical-values-to-binary-using-pandas
data = pd.get_dummies(data)

data.dropna(inplace=True)

data.describe()


,Successful,Industry_Banking/Financial Services,Industry_Computer Systems Design and Services,Industry_Energy,Industry_Fintech,Industry_Government,Industry_Healthcare,Industry_Higher Education,Industry_Insurance,"Industry_Internet, Telecomm or Information Services",Industry_Manufacturing,Industry_Media & Advertising Services,Industry_Other:,Industry_Retail and Consumer Services,Industry_Software Development,"Industry_Transportation, or Supply Chain"
count,2618,2618,2618,2618,2618,2618,2618,2618,2618,2618,2618,2618,2618,2618,2618,2618
unique,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2
top,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
freq,2107,2498,2539,2570,2473,2449,2420,2527,2558,2493,2479,2535,2259,2479,1851,2522


In [31]:
import statsmodels.api as sm

# https://stackoverflow.com/questions/29763620/how-to-select-all-columns-except-one-in-pandas
model = sm.OLS(data["Successful"].to_numpy(), data.loc[:, data.columns != 'Successful'].to_numpy()).fit()

model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.003
Model:                            OLS   Adj. R-squared:                 -0.003
Method:                 Least Squares   F-statistic:                    0.5282
Date:                Tue, 13 May 2025   Prob (F-statistic):              0.918
Time:                        22:40:19   Log-Likelihood:                -1288.2
No. Observations:                2618   AIC:                             2606.
Df Residuals:                    2603   BIC:                             2694.
Df Model:                          14                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x1             0.8083      0.036     22.309      0.000       0.737       0.879
x2             0.7975      0.045     17.858      0.000       0.710       0.885
x3             0.8333      0.057     14.546      0.000       0.721       0.946
x4             0.8000      0.033     24.270      0.000       0.735       0.865
x5             0.7811      0.031     25.581      0.000       0.721       0.841
x6             0.7727      0.028     27.394      0.000       0.717       0.828
x7             0.7912      0.042     19.015      0.000       0.710       0.873
x8             0.8667      0.051     16.913      0.000       0.766       0.967
x9             0.8000      0.036     22.534      0.000       0.730       0.870
x10            0.8201      0.034     24.361      0.000       0.754       0.886
x11            0.7831      0.044     17.975      0.000       0.698       0.869
x12            0.8078      0.021     38.561      0.000       0.767       0.849
x13            0.7842      0.034     23.292      0.000       0.718       0.850
x14            0.8227      0.014     57.402      0.000       0.795       0.851
x15            0.7604      0.041     18.771      0.000       0.681       0.840
==============================================================================
Omnibus:                      575.248   Durbin-Watson:                   2.070
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1037.896
Skew:                          -1.532   Prob(JB):                    4.21e-226
Kurtosis:                       3.364   Cond. No.                         4.00
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

Looking at the R-squared value here, we can obviously see that the industry you choose to work in has effectively no correlation with your level of success in life.  So restricting yourself to specific industries becuase they are more prestigious or because you think you will be more successful there is, in reality, only restricting your possible job opportunities.  

(I know that a linear model is not ideal for this kind of data, but I just wanted to establish a lack of relation between the inputs and outputs.  I don't need to make any predictions or do anything complicated with this model.)

# Analysis of developers in management

In [32]:
# Only using columns that I find relevant for the question I want to ask
data = raw_data.filter(items=[
    "MainBranch",
    "Employment",
    "EdLevel",
    "ICorPM",
    "Country",
    "ConvertedCompYearly",
    "JobSat"
])

# Filtering results for full time, employed developers in the US
data["Country"] = data["Country"] == "United States of America"
data["MainBranch"] = data["MainBranch"] == "I am a developer by profession"
data["Employment"] = data["Employment"] == "Employed, full-time"
data["ICorPM"] = data["ICorPM"].str.contains("manager")

# Removing NA values
data.dropna(inplace=True, subset=[
    "MainBranch",
    "Employment",
    "EdLevel",
    "ICorPM",
    "Country",
    "ConvertedCompYearly",
    "JobSat"
])

# Dollar amount is poverty line for a 4 person family
data["Successful"] = (data["JobSat"] > 5) & (data["ConvertedCompYearly"] > 32150)

# https://stackoverflow.com/questions/17071871/how-do-i-select-rows-from-a-dataframe-based-on-column-values
data = data.loc[data["Country"] & data["MainBranch"] & data["Employment"]]
data = data.filter(items=[
#    "EdLevel",
    "ICorPM",
#    "JobSat"
    "Successful"
])

data.dropna(inplace=True)

data.describe()


,ICorPM,Successful
count,2631,2631
unique,2,2
top,False,True
freq,2374,2117


In [33]:
data.value_counts()

ICorPM  Successful
False   True          1902
        False          472
True    True           215
        False           42
Name: count, dtype: int64

This once again shows that regardless of whether you are an individual contributor or a people manager, there is still roughly a 80% success ratio for software developers.  The ratio for people managers is admittedly slightly higher, at 83.6%, but I'd say that this is within the margin for error with this relatively small amount of people managers in this dataset.  It could also mean that there is a slight positive association between being a manager and being successful, which would also make sense.